# Philip Carr
# CS/CNS/EE_156a_Homework_7_Code_Part_3 (Jupyter Notebook)

Code for The PLA vs. SVM comparisons (Problems 8 - 10)

In [25]:
import random as rn
import numpy as np

# Package imported for SVM: Scikit-Learn sklearn.svm SVC
from sklearn.svm import SVC

In [26]:
def sign(x):
    '''
    Return the sign of a number (1 if positive, 0 if 0, or -1 if
    negative).
    
    Return type: int
    '''
    if x >= 0:
        return 1
    elif x == 0:
        return 0
    else:
        return -1

In [27]:
def target_function_line_value(x_point, y_point, m, b):
    '''
    Return the evaluation of the target function (line with slope m
    and y-intercept b) for a given point (1, x_point, y_point) and
    sign_orientation: determines whether points have value of 1 or -1
    when above the target function line with this value being either
    1 or -1 respectively.
    
    Return type: int
    '''
    y_line = m * x_point + b
    if y_point >= y_line:
        return 1
    else:
        return -1

In [28]:
def get_target_function_line():
    '''
    Return a randomly generated target function line's slope (m) and
    y-intercept (b).
    
    Return type: tuple of floats
    '''
    x1 = rn.uniform(-1, 1)
    y1 = rn.uniform(-1, 1)
    x2 = rn.uniform(-1, 1)
    y2 = rn.uniform(-1, 1)
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return m, b

In [29]:
def get_target_function():
    '''
    Return a randomly generated target function line.
    
    Return type: function that takes two floats and returns an int
    (instance of the target_function_line_value function).
    '''
    m, b = get_target_function_line()
    return m, b, lambda x_point, y_point: \
        target_function_line_value(x_point, y_point, m, b)

In [30]:
def get_random_points(N):
    '''
    Return a list of randomly generated points within
    the region [-1, 1] x [-1, 1].
    
    Return type: list of points (each point is a list)
    '''
    random_points = []
    for n in range(N):
        x0 = 1.0 # artificial coordinate
        x1 = rn.uniform(-1, 1)
        x2 = rn.uniform(-1, 1)
        random_points.append([x0, x1, x2])
    return np.array(random_points, dtype=np.float64)

In [31]:
def get_point_values(points, target_function):
    '''
    Return the values corresponding to the (list of) points given.
    The returned values are function evaluations of the target
    function target_function of the given points.
    
    Return type: list of ints
    '''
    point_values = []
    for i in range(len(points)):
        value = target_function(points[i,1], points[i,2])
        point_values.append(value)
    return np.array(point_values, dtype=np.float64)

In [32]:
class PLA:
    '''
    This class represents the PLA (Perceptron Learning Algorithm).
    This class contains the weights, as well as methods for running
    the Perceptron Learning Algorithm.
    '''
    def __init__(self, n=2):
        '''
        Initialize the weights of the PLA using the given
        dimension n of the points to work with in R^n space.
        
        Return type: class (PLA)
        '''
        self.weights = np.array([0] * (n + 1), dtype=np.float64)
    
    def __repr__(self):
        '''
        Print the weights of the PLA.
        
        Return type: string
        '''
        print("LogReg weights:", self.weights)
    
    def get_weights(self):
        '''
        Return the weights of the PLA.
        
        Return type: array of floats
        '''
        return self.weights
    
    def evaluate(self, point):
        """
        Return a point's value using the PLA's weights.
        """
        real_value = 0
        assert(len(self.weights) == len(point))
        for i in range(len(self.weights)):
            real_value += self.weights[i] * point[i]
            
        return sign(real_value)
    
    def get_misclassified_indices(self, points, values):
        '''
        Return the list of indices of the points list corresponding
        to the points that the PLA misclassifies (when the target
        function and PLA hypothesis function evaluations of a point
        disagree).
        
        Return type: None
        '''
        missclassified_indices = []
        for i in range(len(points)):
            if self.evaluate(points[i]) != values[i]:
                missclassified_indices.append(i)
        
        return missclassified_indices
    
    def get_classification_error(self, points, values):
        """
        Return the classification error given by
        P[f(x) != g_PLA(x)], where f(x) is the target
        function, g_PLA is the hypothesis function of
        the PLA, and P[f(x) != g_PLA(x)] measures the
        probability of disagreement between f and g_PLA
        (classification error).
        """
        misclassified_indices = \
            self.get_misclassified_indices(points, values)
        return np.float64(len(misclassified_indices)) / \
               len(points)
    
    def fit(self, points, values, max_iterations=100):
        '''
        Return the number of iterations it takes to converge the
        PLA to correctly classify all the given points.
        
        Return type: int
        '''
        iteration_count = 0
        
        # Create the list of misclassified point indices.
        misclassified_indices = \
            self.get_misclassified_indices(points, values)
        
        while misclassified_indices != [] and \
              iteration_count < max_iterations:
            
            # Randomly choose an index of a misclassified point.
            misclassified_index = rn.choice(misclassified_indices)
            
            # Iterate the weights using the chosen misclassified.
            # point
            self.weights += values[misclassified_index] \
                            * points[misclassified_index]
            
            # Update the list of misclassified point indices.
            misclassified_indices = \
                self.get_misclassified_indices(points, values)
            
            iteration_count += 1

In [33]:
def PLA_and_SVC_results(N_train):
    """
    Return out-of-sample errors of the PLA (Perceptron Learning
    Algorithm) and SVM (Support Vector Machine) and number of
    support vectors of the SVM that resulted after being trained on
    a randomly generated dataset of size N_train.
    """
    # Initialize training data.
    training_values = np.zeros(N_train)
    training_points = get_random_points(N_train)
    m, b, t_function = get_target_function()
    while len(np.unique(training_values)) == 1:
        training_points = get_random_points(N_train)
        m, b, t_function = get_target_function()
        training_values = get_point_values(training_points,
                                           t_function)
    
    # Initialize and fit PLA to training data.
    pla = PLA()
    pla.fit(training_points, training_values)
    
    # Initialize and fit SVM (Scikit-Learn LinearSVC) to training
    # data
    svc = SVC(C=float("inf"), kernel="linear",
              decision_function_shape="ovr")
    # [:,1:] takes out the x0 = 1 artificial values
    svc.fit(training_points[:,1:], training_values) 
    
    # Initialize out-of-sample data
    N_out_sample = 1000
    out_sample_points = get_random_points(N_out_sample)
    out_sample_values = get_point_values(out_sample_points,
                                         t_function)
    
    # Compute the out-of-sample classification error of the PLA.
    pla_out_sample_error = \
        pla.get_classification_error(out_sample_points,
                                     out_sample_values)
    
    # Compute the out-of-sample classification error of the SVM.
    # [:,1:] takes out the x0 = 1 artificial values
    svc_out_sample_error = 1.0 - \
        svc.score(out_sample_points[:,1:], out_sample_values)
    
    return pla_out_sample_error, svc_out_sample_error, \
           len(svc.support_)

In [34]:
def PLA_and_SVC_test(N_train, trials=1000):
    """
    Return the average (using the given number of trials)
    out-of-sample errors of the PLA (Perceptron Learning Algorithm)
    and SVM (Support Vector Machine) and number of support
    vectors of the SVM that resulted after being trained on randomly
    generated datasets each of size N_train.
    """
    total_svc_accurate_over_pla = 0
    total_svc_n_support_vectors = 0
    
    for i in range(trials):
        pla_out_sample_error, svc_out_sample_error, \
        svc_n_support_vectors = PLA_and_SVC_results(N_train)
        
        total_svc_accurate_over_pla += \
            int(svc_out_sample_error < pla_out_sample_error)
        total_svc_n_support_vectors += svc_n_support_vectors
    
    return total_svc_accurate_over_pla / trials, \
           float(total_svc_n_support_vectors) / trials

For Problem 8

In [35]:
mean_svc_accurate_over_pla, mean_svc_n_support_vectors = \
    PLA_and_SVC_test(N_train=10)

print("Percentage of time g_SVM is better than g_PLA in",
      "approximating f for N = 10:",
      str(100 * mean_svc_accurate_over_pla) + "%")

Percentage of time g_SVM is better than g_PLA in approximating f for N = 10: 62.1%


For Problems 9 and 10

In [36]:
mean_svc_accurate_over_pla, mean_svc_n_support_vectors = \
    PLA_and_SVC_test(N_train=100)

print("Percentage of time g_SVM is better than g_PLA in",
      "approximating f for N = 100:",
      str(100 * mean_svc_accurate_over_pla) + "%")
print("Average number of support vectors of g_SVM for N = 100:",
      mean_svc_n_support_vectors)

Percentage of time g_SVM is better than g_PLA in approximating f for N = 100: 68.8%
Average number of support vectors of g_SVM for N = 100: 2.995
